In [ ]:
import torch
import torchaudio
import torchvision
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import logging
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig ,get_scheduler , AdamW
import pandas as pd
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR
from bitsandbytes.optim import Adam8bit
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from peft import PeftModel, LoraConfig, get_peft_model ,  prepare_model_for_kbit_training

2025-03-22 13:38:34.581819: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-22 13:38:34.581894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-22 13:38:34.583212: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-22 13:38:34.590563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-22 13:38:35.486682: W tensorflow/compiler/tf2

In [ ]:
from huggingface_hub import login

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig ,get_scheduler , AdamW
from datasets import load_dataset
import pandas as pd
from torch.utils.data import Dataset
import evaluate
from torch.optim.lr_scheduler import CosineAnnealingLR
from bitsandbytes.optim import Adam8bit
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from peft import PeftModel, LoraConfig, get_peft_model ,  prepare_model_for_kbit_training
import logging


lora_config = LoraConfig(
    r=32,
    lora_alpha=64 , 
    target_modules = ["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"  
)

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  
    llm_int8_threshold=6.0,  # determine threshold for dynamic quantization
    device_map= {"": 0}
)
model_name = "google/mt5-base" # load model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=quantization_config,device_map={"": 0})

model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


trainable params: 3,538,944 || all params: 585,940,224 || trainable%: 0.6039769681352343


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 768)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 768)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): Linear4bit(
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): ParameterDict()
                    (base_layer): Linea

In [ ]:

print(f"model size after Apply Quantization: {model.get_memory_footprint() / 1e9:.2f} GB")


In [ ]:

#load and prepare data
def load_and_prepare_data():
    dataset = load_dataset("pythainlp/thaisum")

    df_train_50k = dataset["train"].to_pandas().sample(n=50000, random_state=42) # use only 50,000 samples

    df1_train = df_train_50k.sample(n=6700, random_state=42) # only 6700 because of limitation of VRAM
    df_remaining = df_train_50k.drop(df1_train.index)

    df_train = df1_train
    df_valid = dataset["validation"].to_pandas()  # validation set
    df_test = dataset["test"].to_pandas()


    print(df_train.head())
    valid_size = int(df_train.shape[0] * 0.1)  # use 10% of train set
    test_size = int(df_train.shape[0] * 0.1)

    # 
    df_valid = df_valid.sample(n=valid_size, random_state=42)
    df_test = df_test.sample(n=test_size, random_state=42)

    return df_train, df_valid, df_test

class TextSummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_length=1024):
    # def __init__(self, texts, summaries, tokenizer, max_length=1024):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        input_text = str(self.texts[idx])
        target_text = str(self.summaries[idx])

        input_encoding = self.tokenizer(
            input_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt"
        )

        labels = target_encoding.input_ids.squeeze(0) # Ignore padding during computation.
        labels[labels == 0] = -100 

        return {
            "input_ids": input_encoding.input_ids.squeeze(0),
            "attention_mask": input_encoding.attention_mask.squeeze(0),
            "labels": labels
        }
df_train, df_valid, df_test = load_and_prepare_data()

# build dataset
train_dataset = TextSummaryDataset(df_train["body"].tolist(), df_train["summary"].tolist(), tokenizer)
valid_dataset = TextSummaryDataset(df_valid["body"].tolist(), df_valid["summary"].tolist(), tokenizer)
test_dataset = TextSummaryDataset(df_test["body"].tolist(), df_test["summary"].tolist(), tokenizer)


                                                    title  \
88734   ฟู้ดทรัคไฟรั่ว ช็อตสาวสิ้นใจ กฟน.แจงแรงดันเพีย...   
333624  องค์กรผู้บริโภคโวย กทค.แอบทำมติลับ พรีเพด เอื้...   
117808                          ห้ามวัดกทม.สร้างถาวรวัตถุ   
143066                 2วายร้ายชิง4.2แสนมือเปล่าบุกธนาคาร   
274333  จาย นำทัพสู้ศึกชิงแชมป์เอเชีย ล่าแต้มสะสมโอลิมปิก   

                                                     body  \
88734   เมื่อเวลา 21.00 น.วันที่ 2 ก.ค.62 พ.ต.ท.รักเกี...   
333624   พร้อมสับองค์กรโทรคมนาคมเผยแพร่มติผ่านเว็บไซต์...   
117808  เจ้าคณะใหญ่หนกลางสั่ง จวกกาฝากทําลายศาสนา,เจ้า...   
143066  แล้วหนีออกมาขึ้นแท็กซี่หน้าแบงก์ ตบบ่าลูกค้าพู...   
274333   หากเก็บเพิ่มได้ 600-800 คะแนน โอกาสที่จะได้โค...   

                                                  summary                type  \
88734   เร่งหาสาเหตุที่แท้จริง สาววัย 24 ปี มานั่งกินก...      ข่าว,อาชญากรรม   
333624  สหพันธ์องค์กรผู้บริโภค เรียกร้อง กทค.แจงเหตุมี...  คุณภาพชีวิต,ไอซีที   
117808  เจ้าคณะใหญ่หนกล

In [ ]:

# สร้าง training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results', 
    num_train_epochs=18, 
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    learning_rate=5e-5,
    eval_accumulation_steps=2,
    logging_dir='./logs',
    evaluation_strategy="steps",  
    save_strategy="steps", 
    warmup_steps = 500, 
    eval_steps=100,
    save_steps=100,  
    save_total_limit=2, 
    weight_decay=0.01,
    logging_steps=10,
    disable_tqdm=False,
    report_to="none",
    bf16=True,
    lr_scheduler_type='cosine_with_restarts', # avoid local minima
    predict_with_generate=True,
    optim="paged_adamw_8bit",  # Using AdamW optimizer from torch
    load_best_model_at_end=True, 
    gradient_accumulation_steps = 8, # reduce vram


)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  
)



# Train model 
for name, param in model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

project = "mt5base-thesis-finetune-"
run_name = 'train-model-dir-round-5'
output_dir = "./" + project + run_name

trainer.train()
model.config._name_or_path = output_dir
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
base_model = model.base_model  
base_model.save_pretrained(output_dir)  
trainer.save_model(output_dir) 
model.config.save_pretrained(output_dir)  
base_model.config.save_pretrained(output_dir)  

eval_results = trainer.evaluate()

In [ ]:
model_path = "./mt5base-thesis-finetune-train-model-dir-round-5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(model)
print(tokenizer)

In [ ]:

#  ROUGE score calculation
rouge = evaluate.load("rouge")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def evaluate_model(model, tokenizer, dataset, num_samples):
    model.eval() 
    preds, refs = [], []
    
    for i, sample in enumerate(dataset):
        if i >= num_samples: 
            break
        
        input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
        reference_summary = tokenizer.decode(
            [token for token in sample["labels"] if token != -100], skip_special_tokens=True
        )
        
        
        inputs = {
            "input_ids": sample["input_ids"].unsqueeze(0).to(device),
            "attention_mask": sample["attention_mask"].unsqueeze(0).to(device)
        }
        
        # Generate summary with optimized parameters
        with torch.no_grad():output_tokens = model.generate(**inputs, max_length=1000, num_beams=5)
        
        # Decode summary output and append
        predicted_summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

        preds.append(predicted_summary)
        refs.append(reference_summary)
    
    # คำนวณ ROUGE score
    rouge_scores = rouge.compute(predictions=preds, references=refs)
    
    return rouge_scores

# ROUGE Score
rouge_results = evaluate_model(model, tokenizer, valid_dataset, num_samples=200)

print("ROUGE Score:", rouge_results)


In [ ]:
# real test 
def summarize_text(text, model, tokenizer, max_input_length=3000, max_output_length=1500):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_input_length)
    
    # Move tensors to the same device as the model
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    summary_ids = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=max_output_length,
    min_length=200,
    num_beams=5,             
    do_sample=True,        
    length_penalty=1.1,      
    no_repeat_ngram_size=3,  
    repetition_penalty=1.1,   
    top_p=0.95,               
    temperature=0.9,
    early_stopping=True       
    )


    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
sample_text = """ประเทศไทย- หรือชื่อทางการว่า ราชอาณาจักรไทย เป็นรัฐชาติอันตั้งอยู่บนคาบสมุทรอินโดจีนและมลายู ในภูมิภาคเอเชียตะวันออกเฉียงใต้ มีพรมแดนด้านตะวันออกติดประเทศลาวและประเทศกัมพูชา ทิศใต้เป็นแดนต่อแดนประเทศมาเลเซียและอ่าวไทย ทิศตะวันตกติดทะเลอันดามันและประเทศพม่า และทิศเหนือติดประเทศพม่าและลาว มีแม่น้ำโขงกั้นเป็นบางช่วง ปกครองด้วยระบอบประชาธิปไตยแบบมีรัฐสภา มีศูนย์กลางการบริหารราชการแผ่นดินอยู่ที่กรุงเทพมหานคร และการปกครองส่วนภูมิภาค จัดระเบียบเป็น 76 จังหวัด
ประเทศไทยมีขนาดใหญ่เป็นอันดับที่ 50 ของโลก มีเนื้อที่ 513,115 ตารางกิโลเมตร และมีประชากรมากเป็นอันดับที่ 20 ของโลก คือ ประมาณ 66 ล้านคน กับทั้งยังเป็นประเทศอุตสาหกรรมใหม่ โดยมีรายได้หลักจากภาคอุตสาหกรรมและการบริการ ไทยมีแหล่งท่องเที่ยวที่มีชื่อเสียงเป็นอันมาก อาทิ พัทยา, ภูเก็ต, กรุงเทพมหานคร และเชียงใหม่ ซึ่งสร้างรายได้ให้แก่ประเทศ เช่นเดียวกับการส่งออกอันมีส่วนสำคัญในการพัฒนาเศรษฐกิจและด้วยจีดีพีของประเทศ ซึ่งมีมูลค่าราว 334,026 ล้านดอลลาร์สหรัฐ ตามที่ประมาณใน พ.ศ. 2553 เศรษฐกิจของประเทศไทยนับว่าใหญ่เป็นอันดับที่ 30 ของโลก
ในอาณาเขตประเทศไทย พบหลักฐานของมนุษย์ซึ่งมีอายุเก่าแก่ที่สุดถึงห้าแสนปี นักประวัติศาสตร์มักถือว่าอาณาจักรสุโขทัยเป็นจุดเริ่มต้นของประวัติศาสตร์ไทย ซึ่งต่อมาตกอยู่ในอิทธิพลของอาณาจักรอยุธยา อันมีความยิ่งใหญ่กว่า และมีการติดต่อกับชาติตะวันตก อาณาจักรอยุธยามีอายุยืนยาว 417 ปีก็เสื่อมอำนาจและล่มสลายไปโดยสิ้นเชิง สมเด็จพระเจ้ากรุงธนบุรีทรงกอบกู้เอกราชและสถาปนาอาณาจักรธนบุรี เหตุการณ์ความวุ่นวายในช่วงปลายอาณาจักร นำไปสู่ยุคสมัยของราชวงศ์จักรีแห่งกรุงรัตนโกสินทร์
ช่วงต้นกรุง ประเทศเผชิญภัยคุกคามจากชาติใกล้เคียง แต่หลังรัชสมัยพระบาทสมเด็จพระจอมเกล้าเจ้าอยู่หัวเป็นต้นมา ชาติตะวันตกเริ่มมีอิทธิพลในภูมิภาคเป็นอย่างมาก นำไปสู่การเข้าเป็นภาคีแห่งสนธิสัญญาหลายฉบับ และการเสียดินแดนบางส่วน กระนั้น ไทยก็ยังธำรงตนมิได้เป็นอาณานิคมของชาติใด ๆ ต่อมาจนช่วงสงครามโลกครั้งที่หนึ่ง ไทยได้เข้าร่วมกับฝ่ายสัมพันธมิตร และในปี พ.ศ. 2475 ได้มีการปฏิวัติเปลี่ยนแปลงจากระบอบสมบูรณาญาสิทธิราชมาเป็นประชาธิปไตย และไทยได้เข้ากับฝ่ายอักษะในระหว่างสงครามโลกครั้งที่สอง จนช่วงสงครามเย็น ไทยได้ดำเนินนโยบายเป็นพันธมิตรกับสหรัฐอเมริกา ทหารเข้ามามีบทบาทในการเมืองไทยอย่างมากหลังปฏิวัติสยามอยู่หลายสิบปี กระทั่งมีการตั้งรัฐบาลพลเรือน และเข้าสู่ยุคโลกเสรีในปัจจุบัน"""
print("🔹 ข้อความต้นฉบับ: ", sample_text)
print("🔹 ข้อความสรุป:", summarize_text(sample_text, model, tokenizer))